In [1]:
import pandas as pd
import numpy as np
import os
import time

from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler

# Load Dataset

In [55]:
os.getcwd()

'/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/jfschulz_notebooks'

In [50]:
path = "/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data"
file = "Glioma-clinic-TCGA-proteins.csv" 
df = pd.read_csv(f"{path}/{file}", header=0)
#df = pd.read_csv(path)
df.head(2)

,Case,years_to_birth,gender,histological_type,race,ethnicity,radiation_therapy,Grade,Mutation.Count,Percent.aneuploidy,...,p27_p,p27_pT157_p,p27_pT198_p,p38_pT180_Y182_p,p53_p,p62-LCK-ligand_p,p70S6K_p,p70S6K_pT389_p,p90RSK_p,p90RSK_pT359_S363_p
0,TCGA-CS-4938,31,female,astrocytoma,white,not hispanic or latino,no,G2,15,0.069412,...,-0.425127,-0.033398,0.289192,1.060163,-0.407456,-0.470354,-0.107559,-0.060441,-0.053104,-0.011132
1,TCGA-CS-6665,51,female,astrocytoma,white,not hispanic or latino,yes,G3,75,0.524814,...,0.076536,0.011809,-0.047973,-0.054275,-0.244020,0.106282,-0.034369,0.585072,0.431720,-0.201412


In [5]:
#df.info(verbose=True, show_counts=True)
df["histological_type"].unique()

array(['astrocytoma', 'oligoastrocytoma', 'oligodendroglioma'],
      dtype=object)

# Define Features and Target

In [51]:
# Define target and features
cols_to_drop = ['Case', 'years_to_birth', 'gender', 'histological_type', 'race',
       'ethnicity', 'radiation_therapy', 'Grade', 'Mutation.Count',
       'Percent.aneuploidy', 'IDH.status', 'outcome']
X = df.drop(cols_to_drop, axis = 1)
y = df["outcome"]

X.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 174 columns):
 #    Column                     Non-Null Count  Dtype  
---   ------                     --------------  -----  
 0    14-3-3_beta_p              306 non-null    float64
 1    14-3-3_epsilon_p           306 non-null    float64
 2    14-3-3_zeta_p              306 non-null    float64
 3    4E-BP1_p                   306 non-null    float64
 4    4E-BP1_pS65_p              306 non-null    float64
 5    4E-BP1_pT37_T46_p          306 non-null    float64
 6    4E-BP1_pT70_p              306 non-null    float64
 7    53BP1_p                    306 non-null    float64
 8    A-Raf_pS299_p              306 non-null    float64
 9    ACC1_p                     306 non-null    float64
 10   ACC_pS79_p                 306 non-null    float64
 11   ACVRL1_p                   306 non-null    float64
 12   AMPK_alpha_p               306 non-null    float64
 13   AMPK_pT172_p               306 no

In [52]:
# Create train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [53]:
X_test

,14-3-3_beta_p,14-3-3_epsilon_p,14-3-3_zeta_p,4E-BP1_p,4E-BP1_pS65_p,4E-BP1_pT37_T46_p,4E-BP1_pT70_p,53BP1_p,A-Raf_pS299_p,ACC1_p,...,p27_p,p27_pT157_p,p27_pT198_p,p38_pT180_Y182_p,p53_p,p62-LCK-ligand_p,p70S6K_p,p70S6K_pT389_p,p90RSK_p,p90RSK_pT359_S363_p
277,-0.046266,0.010482,0.005867,-0.128982,-0.097890,-0.286694,-0.095360,-0.315572,-0.079862,-0.190765,...,0.282454,0.215619,0.049541,0.394208,-0.173627,-0.088369,-0.160519,0.494404,0.412308,-0.081711
53,0.062830,0.341067,0.509924,-0.097006,0.050285,-0.555042,-0.108214,-0.047927,-0.029663,0.203558,...,0.233966,0.165461,0.225253,-0.065668,-0.268971,-0.059341,-0.237159,-0.004258,0.383249,-0.383407
228,0.005718,-0.049226,-0.176334,0.110042,0.084026,0.205933,-0.054664,0.562995,0.063561,0.578327,...,0.361402,-0.143755,-0.065628,0.216761,0.033749,0.105418,-0.248534,-0.251022,0.017531,-0.108484
11,0.070416,0.068269,0.534429,-0.403894,0.106003,-0.169416,-0.234652,-0.223324,0.019363,0.397784,...,0.113838,-0.081752,-0.028470,0.059929,-0.093907,0.007829,-0.040814,0.726636,0.156300,-0.195128
120,-0.063005,0.047834,0.414731,-0.374914,0.616939,0.810131,-0.173296,-0.381965,0.101445,0.273052,...,-0.273558,-0.042558,-0.142239,-0.141052,-0.224926,-0.073704,-0.076397,1.391063,0.137433,0.104366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,0.163980,0.029452,0.649739,-0.445981,1.062340,1.745698,0.025130,-0.152001,0.239102,0.431975,...,-0.104785,0.252492,-0.155413,-0.746732,0.029633,0.219986,0.093239,2.279390,0.222581,0.341038
59,-0.029360,-0.020617,-0.058948,0.173444,0.115165,0.235074,0.065090,-0.202499,0.023789,0.080197,...,0.422843,-0.034226,0.087660,-0.013897,0.336788,0.012812,0.066353,-0.132282,-0.191353,0.017849
99,-0.082008,0.253038,0.390207,-0.166794,-0.119515,-0.566232,-0.140786,-0.212030,0.160434,0.107996,...,-0.079885,0.121889,0.062165,-0.484047,0.274336,0.177687,0.064581,1.263077,0.215282,0.108698
146,0.058965,-0.050453,0.028261,-0.050225,-0.221785,-0.788873,-0.199018,-0.101192,0.108723,-0.416462,...,-0.063589,-0.046370,0.004108,-0.000405,0.283721,0.420645,-0.148972,-0.176567,-0.095915,0.066603


In [54]:
# save X_test data set to use for API testing:
path = "/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data"
file = "Glioma-clinic-TCGA-proteins-test-with-identifier.csv" 

X_test.to_csv(path_or_buf=f"{path}/{file}", header=True, index=True, index_label="Identifier")

In [9]:
# save one row of df data set to use for API testing: row 85 --> outcome = 1
path = "/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data"
file = "Glioma-clinic-TCGA-proteins-test-with-identifier-outcome1.csv" 
cols_to_drop = ['Case', 'years_to_birth', 'gender', 'histological_type', 'race',
       'ethnicity', 'radiation_therapy', 'Grade', 'Mutation.Count',
       'Percent.aneuploidy', 'IDH.status', 'outcome']

tmp1 = df[85:86].drop(cols_to_drop, axis = 1)
tmp1.to_csv(path_or_buf=f"{path}/{file}", header=True, index=True, index_label="Identifier")

In [10]:
# save one row of df data set to use for API testing: row 274 --> outcome = 0
path = "/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data"
file = "Glioma-clinic-TCGA-proteins-test-with-identifier-outcome0.csv" 
cols_to_drop = ['Case', 'years_to_birth', 'gender', 'histological_type', 'race',
       'ethnicity', 'radiation_therapy', 'Grade', 'Mutation.Count',
       'Percent.aneuploidy', 'IDH.status', 'outcome']

tmp1 = df[274:275].drop(cols_to_drop, axis = 1)
tmp1.to_csv(path_or_buf=f"{path}/{file}", header=True, index=True, index_label="Identifier")

In [11]:
df[274:275]["outcome"]

274    0
Name: outcome, dtype: int64

## Code to create feature table based on uploaded csv file for api

In [12]:
# Code to create 
tmp = pd.read_csv(f"{path}/{file}", header=0)
tmp.head(3)

if 'Identifier' in tmp.columns:
    X_pred = tmp.drop(["Identifier"], axis = 1)
else:
    X_pred = tmp

# Preprocess

In [13]:
# Scale
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(X_train)
X_train_proc = min_max_scaler.transform(X_train)
X_test_proc = min_max_scaler.transform(X_test)

## Functions to save and load scaler for api

In [14]:
# Save scaler

from joblib import dump, load
import time


def save_scaler(scaler_to_save = None, 
               scaler_type = None, 
               path_to_save = "/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_scalers"
              ):
    """
    Persist trained model locally on the hard drive at f"{path_to_save/scaler_type/f"{timestamp}.joblib"
    """

    timestamp = time.strftime("%Y%m%d-%H%M%S")
    
    # Save scaler locally
    scaler_path_file = os.path.join(f"{path_to_save}/{scaler_type}_{timestamp}.joblib")
   
    dump(scaler_to_save, scaler_path_file) 

    print(f"✅ Scaler saved locally at {scaler_path_file}")
    
# Load scaler
def load_scaler(path = '/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_scalers',
               file = 'MinMax_20240306-102844.joblib'
              ):
    path_file = f"{path}/{file}"

    scaler = load(path_file) 
    return scaler

In [15]:
save_scaler(scaler_to_save= min_max_scaler,
           scaler_type="MinMax"
           )

✅ Scaler saved locally at /home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_scalers/MinMax_20240307-101324.joblib


In [16]:
load_scaler(file ="MinMax_20240306-102844.joblib")

/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


MinMaxScaler()

# Build Model

In [17]:
# Initialize model
model = SGDClassifier(loss = "log_loss")

# Cross validte model performance
np.mean(cross_val_score(model, X_train_proc, y_train, cv=5))

# Train model
history = model.fit(X_train_proc, y_train)

## Functions to save and load model

In [18]:
# Save model

from joblib import dump, load
import time


def save_model(model_to_save = None, 
               model_type = None, 
               path_to_save = "/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_models"
              ):
    """
    Persist trained model locally on the hard drive at f"{path_to_save/model_type/f"{timestamp}.joblib"
    """

    timestamp = time.strftime("%Y%m%d-%H%M%S")
    
    # Save model locally
    model_path_file = os.path.join(f"{path_to_save}/{model_type}_{timestamp}.joblib")
   
    dump(model_to_save, model_path_file) 

    print(f"✅ Model saved locally at {model_path_file}")

In [19]:
save_model(model_to_save = model, model_type = "SGDclassifier")

✅ Model saved locally at /home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_models/SGDclassifier_20240307-101327.joblib


In [20]:
# Load model
def load_model(path = '/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_models',
               file = 'SGDclassifier_20240305-135742.joblib'
              ):
    path_file = f"{path}/{file}"

    model = load(path_file) 
    return model

In [21]:
model3 = load_model()
scaler = load_scaler()


/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SGDClassifier from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [22]:
X_test.shape
X_test_proc = scaler.transform(X_test)

In [23]:
X_test_proc.shape

(62, 174)

In [24]:
model3.predict(X_test_proc).shape

(62,)

In [25]:
pd.DataFrame(model.predict(X_test_proc), columns=["Outcome"], dtype = int)

,Outcome
0,0
1,0
2,0
3,0
4,0
...,...
57,0
58,0
59,0
60,1


In [26]:
pd.DataFrame(model.predict_proba(X_test_proc), columns=["Probability_0", "Probability_1"], dtype = float)


,Probability_0,Probability_1
0,1.000000,1.539120e-09
1,0.999490,5.100199e-04
2,1.000000,4.376320e-10
3,0.817148,1.828519e-01
4,1.000000,3.463690e-08
...,...,...
57,0.999639,3.609115e-04
58,1.000000,4.839377e-10
59,1.000000,4.390907e-17
60,0.000056,9.999436e-01


In [27]:
outcome = pd.DataFrame(model.predict(X_test_proc), columns=["Outcome"], dtype = int)
prob = pd.DataFrame(model.predict_proba(X_test_proc), columns=["Probability_0", "Probability_1"], dtype = float)

    # Merge results into dataframe
result = pd.merge(prob,outcome, left_index=True, right_index=True)


In [28]:
{
    "Outcome": result["Outcome"].tolist,
    "Probability_0": result["Probability_0"].tolist()
    "Probability_1": result["Probability_1"].tolist()
}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3599829651.py, line 3)

In [29]:
{k: v.tolist() for k, v in result.iterrows()}


{0: [0.9999999984608803, 1.539119684558466e-09, 0.0],
 1: [0.9994899801104767, 0.0005100198895232571, 0.0],
 2: [0.999999999562368, 4.376320422855559e-10, 0.0],
 3: [0.8171480614345283, 0.18285193856547163, 0.0],
 4: [0.9999999653631049, 3.4636895143770705e-08, 0.0],
 5: [0.0007739546753290361, 0.999226045324671, 1.0],
 6: [0.9999999999585548, 4.1445134143151145e-11, 0.0],
 7: [1.0, 4.0370873986584997e-19, 0.0],
 8: [4.553750709845872e-10, 0.9999999995446249, 1.0],
 9: [0.9999999999999996, 4.020988942126253e-16, 0.0],
 10: [0.999999999963069, 3.693097321429898e-11, 0.0],
 11: [0.9999999999943312, 5.668799447208983e-12, 0.0],
 12: [1.2311417829646842e-06, 0.999998768858217, 1.0],
 13: [3.4369593193694925e-07, 0.9999996563040681, 1.0],
 14: [0.9999999999999998, 1.6949150987879817e-16, 0.0],
 15: [0.9999998829213026, 1.1707869740514118e-07, 0.0],
 16: [0.9999999999999384, 6.162935390664951e-14, 0.0],
 17: [0.9999999999999486, 5.1415690101531494e-14, 0.0],
 18: [0.9999999999999934, 6.49616

In [30]:
result

,Probability_0,Probability_1,Outcome
0,1.000000,1.539120e-09,0
1,0.999490,5.100199e-04,0
2,1.000000,4.376320e-10,0
3,0.817148,1.828519e-01,0
4,1.000000,3.463690e-08,0
...,...,...,...
57,0.999639,3.609115e-04,0
58,1.000000,4.839377e-10,0
59,1.000000,4.390907e-17,0
60,0.000056,9.999436e-01,1


In [31]:
result_dict = result.to_dict('series')
type(result_dict["Probability_0"])

pandas.core.series.Series

In [32]:
type(list(model.predict(X_test_proc)))

list

In [33]:
# Predict test data
outcome = pd.DataFrame(model.predict(X_test_proc), columns=["Outcome"])
prob = pd.DataFrame(model.predict_proba(X_test_proc), columns=["Probability_0", "Probability_1"])

# Merge results into dataframe
result = pd.merge(prob,outcome, left_index=True, right_index=True)

# Create prediction endpoint for API

## User can upload the data using the upload button

## User has to input the path and the file name to upload file to predict

### Prediction endpoint for one line of prediction

In [34]:
def predict_one(
            path: str, #"/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data"
            file: str, # "Glioma-clinic-TCGA-proteins-test-with-identifier.csv"
            ):
    """
    Make a prediction for every row in your dataset.
    Input needs to be a csv file with rows = samples and columns = proteins.
    The first row has to contain the protein names or any other identifier that will serve as a header.
    If your file has a sample identifier column, name this column "Identifier"
    """

    df = pd.read_csv(f"{path}/{file}", header=0)

    if 'Identifier' in df.columns:
        X_pred = df.drop(["Identifier"], axis = 1)
    else:
        X_pred = df


    # Preprocess
    # Load scaler
    scaler = load_scaler(path = '/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_scalers',
                         file = 'MinMax_20240306-102844.joblib'
                        )

    X_pred_proc = scaler.transform(X_pred)

    # Predict data
    model = load_model()

    outcome_num = int(model.predict(X_pred_proc)[0])
    if outcome_num == 0:
        outcome = "good cancer"
        probability = round(float(model.predict_proba(X_pred_proc)[0][0]), 10)
    else:
        outcome = "bad cancer"
        probability = round(float(model.predict_proba(X_pred_proc)[0][1]), 10)

    return {
                "Outcome": outcome,
                "Probability": probability
            }

In [35]:
predict_one(path = "/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data",
       file = "Glioma-clinic-TCGA-proteins-test-with-identifier-outcome1.csv")

/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SGDClassifier from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'Outcome': 'bad cancer', 'Probability': 1.0}

In [36]:
model = load_model()

/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SGDClassifier from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [37]:
tmp1 = df[274:275].drop(cols_to_drop, axis = 1)
outcome_num = int(model.predict(tmp1)[0])
if outcome_num == 0:
    outcome = "good cancer"
    probability = round(float(model.predict_proba(tmp1)[0][0]), 4)
else:
    outcome = "bad cancer"
    probability = round(float(model.predict_proba(tmp1)[0][1]), 4)

/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(
/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


In [38]:
{
    "Outcome": outcome,
    "Probability": probability
}

{'Outcome': 'good cancer', 'Probability': 1.0}

In [39]:
prob = float(model.predict_proba(tmp1)[0][1])
prob

/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


1.180455213004863e-08

###  Prediction endpoint that should return a dataframe in case the user wants to do several predictions

In [40]:
# http://127.0.0.1:8000/predict?
# path=/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data
# &file=Glioma-clinic-TCGA-proteins-test-with-identifier.csv
@app.get("/predict")
def predict(
            path: str, #"/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data"
            file: str, # "Glioma-clinic-TCGA-proteins-test-with-identifier.csv"
            ):
    """
    Make a prediction for every row in your dataset.
    Input needs to be a csv file with rows = samples and columns = proteins.
    The first row has to contain the protein names or any other identifier that will serve as a header.
    If your file has a sample identifier column, name this column "Identifier"
    """

    df = pd.read_csv(f"{path}/{file}", header=0)

    if 'Identifier' in df.columns:
        X_pred = df.drop(["Identifier"], axis = 1)
    else:
        X_pred = df


    # Preprocess
    # Load scaler
    scaler = load_scaler(path = '/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_scalers',
                         file = 'MinMax_20240306-102844.joblib'
                        )

    X_pred_proc = scaler.transform(X_pred)

    # Predict data
    model = load_model()

    outcome = pd.DataFrame(model.predict(X_pred_proc), columns=["Outcome"], dtype = int)
    prob = pd.DataFrame(model.predict_proba(X_pred_proc), columns=["Probability_0", "Probability_1"], dtype = float)

    # Merge results into dataframe
    result = pd.merge(prob,outcome, left_index=True, right_index=True)
    result_dict = result.to_dict('series')

    return {k: v.tolist() for k, v in result.iterrows()}

    #return {"Outcome": result["outcome"][1]}



In [41]:
result = predict(path = "/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data",
       file = "Glioma-clinic-TCGA-proteins-test-with-identifier.csv")

/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SGDClassifier from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


##  Try to convert dataframe to be predicted to dictionary to pass into the api

In [63]:
# load data to predict
path = "/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data"
file = "Glioma-clinic-TCGA-proteins-test-with-identifier.csv"

df1 = pd.read_csv(f"{path}/{file}", header=0)

if 'Identifier' in df1.columns:
    X_pred = df1.drop(["Identifier"], axis = 1)
else:
    X_pred = df1

In [64]:
X_pred

,14-3-3_beta_p,14-3-3_epsilon_p,14-3-3_zeta_p,4E-BP1_p,4E-BP1_pS65_p,4E-BP1_pT37_T46_p,4E-BP1_pT70_p,53BP1_p,A-Raf_pS299_p,ACC1_p,...,p27_p,p27_pT157_p,p27_pT198_p,p38_pT180_Y182_p,p53_p,p62-LCK-ligand_p,p70S6K_p,p70S6K_pT389_p,p90RSK_p,p90RSK_pT359_S363_p
0,-0.046266,0.010482,0.005867,-0.128982,-0.097890,-0.286694,-0.095360,-0.315572,-0.079862,-0.190765,...,0.282454,0.215619,0.049541,0.394208,-0.173627,-0.088369,-0.160519,0.494404,0.412308,-0.081711
1,0.062830,0.341067,0.509924,-0.097006,0.050285,-0.555042,-0.108214,-0.047927,-0.029663,0.203558,...,0.233966,0.165461,0.225253,-0.065668,-0.268971,-0.059341,-0.237159,-0.004258,0.383249,-0.383407
2,0.005718,-0.049226,-0.176334,0.110042,0.084026,0.205933,-0.054664,0.562995,0.063561,0.578327,...,0.361402,-0.143755,-0.065628,0.216761,0.033749,0.105418,-0.248534,-0.251022,0.017531,-0.108484
3,0.070416,0.068269,0.534429,-0.403894,0.106003,-0.169416,-0.234652,-0.223324,0.019363,0.397784,...,0.113838,-0.081752,-0.028470,0.059929,-0.093907,0.007829,-0.040814,0.726636,0.156300,-0.195128
4,-0.063005,0.047834,0.414731,-0.374914,0.616939,0.810131,-0.173296,-0.381965,0.101445,0.273052,...,-0.273558,-0.042558,-0.142239,-0.141052,-0.224926,-0.073704,-0.076397,1.391063,0.137433,0.104366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,0.163980,0.029452,0.649739,-0.445981,1.062340,1.745698,0.025130,-0.152001,0.239102,0.431975,...,-0.104785,0.252492,-0.155413,-0.746732,0.029633,0.219986,0.093239,2.279390,0.222581,0.341038
58,-0.029360,-0.020617,-0.058948,0.173444,0.115165,0.235074,0.065090,-0.202499,0.023789,0.080197,...,0.422843,-0.034226,0.087660,-0.013897,0.336788,0.012812,0.066353,-0.132282,-0.191353,0.017849
59,-0.082008,0.253038,0.390207,-0.166794,-0.119515,-0.566232,-0.140786,-0.212030,0.160434,0.107996,...,-0.079885,0.121889,0.062165,-0.484047,0.274336,0.177687,0.064581,1.263077,0.215282,0.108698
60,0.058965,-0.050453,0.028261,-0.050225,-0.221785,-0.788873,-0.199018,-0.101192,0.108723,-0.416462,...,-0.063589,-0.046370,0.004108,-0.000405,0.283721,0.420645,-0.148972,-0.176567,-0.095915,0.066603


In [68]:
X_pred_list = {k: v.tolist() for k, v in X_pred.iterrows()}

In [72]:
print(X_pred_list)

{0: [-0.046265886, 0.010482387, 0.005867445, -0.128982268, -0.097889666, -0.286694285, -0.095360044, -0.315571826, -0.079861525, -0.190764932, -0.238326955, -0.048540484, -0.002730457, -0.075113939, 0.029746211, -0.332981871, -0.368910452, -0.0524449, 0.287860131, 0.488310645, 0.206614825, -0.371954514, 0.02017156, -0.007428026, -0.164302779, -0.189112674, -0.177334949, 0.105022221, 0.547064077, -0.088799185, 0.0, 0.02225688, -0.099785565, 0.087806221, 0.120169961, -0.203541575, 0.597637885, -0.064870281, -0.006525207, 0.493392293, -0.289667328, 0.394968561, 0.061731113, 0.166321033, 0.035140285, -0.121493066, -0.309331731, 1.047156113, -0.384492995, 0.068288443, 0.15988954, -0.086162623, -0.591365353, -0.348480285, -0.242205473, -0.036207938, -0.283215889, -0.002190084, 0.183297849, 0.274069847, 0.017332449, -0.171258421, 0.03426366, -0.633766774, -0.286603642, -0.099785565, 0.275832582, 0.331906664, -0.372040393, 0.050319919, 0.042505568, 0.262618311, 0.014002605, -0.401017263, 0.409

In [71]:
pd.DataFrame(X_pred_list)

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,-0.046266,0.062830,0.005718,0.070416,-0.063005,0.090642,-0.093869,0.117048,-0.036813,-0.188983,...,0.129091,0.188977,-0.092673,-0.070395,0.088773,0.163980,-0.029360,-0.082008,0.058965,-0.089893
1,0.010482,0.341067,-0.049226,0.068269,0.047834,0.127904,-0.252845,0.241715,-0.047815,-0.311200,...,0.004169,-0.058367,-0.135226,-0.034295,0.054260,0.029452,-0.020617,0.253038,-0.050453,0.184818
2,0.005867,0.509924,-0.176334,0.534429,0.414731,0.339170,0.410162,0.206968,-0.122072,-0.225733,...,0.098866,-0.111915,-0.120118,0.054421,-0.140119,0.649739,-0.058948,0.390207,0.028261,-0.379278
3,-0.128982,-0.097006,0.110042,-0.403894,-0.374914,-0.150820,-0.327964,0.007823,-0.050649,0.342750,...,-0.162787,-0.212003,0.220188,-0.266592,0.037902,-0.445981,0.173444,-0.166794,-0.050225,-0.113470
4,-0.097890,0.050285,0.084026,0.106003,0.616939,0.365706,0.197215,-0.062358,-0.220442,0.241693,...,0.229510,-0.164837,0.106521,-0.158487,-0.313040,1.062340,0.115165,-0.119515,-0.221785,-0.381345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,-0.088369,-0.059341,0.105418,0.007829,-0.073704,0.004746,0.340390,0.035733,0.250815,-0.466182,...,0.087680,-0.095666,-0.474115,0.043443,0.254037,0.219986,0.012812,0.177687,0.420645,-0.268641
170,-0.160519,-0.237159,-0.248534,-0.040814,-0.076397,-0.039065,0.020190,-0.045691,-0.048886,0.247032,...,-0.100576,-0.122288,0.116780,-0.043661,-0.230895,0.093239,0.066353,0.064581,-0.148972,-0.000092
171,0.494404,-0.004258,-0.251022,0.726636,1.391063,2.322902,0.183455,0.030409,0.389122,-0.111868,...,1.727192,-0.123695,-0.503557,0.325470,-0.116572,2.279390,-0.132282,1.263077,-0.176567,-1.126171
172,0.412308,0.383249,0.017531,0.156300,0.137433,0.151344,0.125674,0.030682,-0.001712,-0.047198,...,0.156863,-0.164060,-0.039171,0.111535,-0.196836,0.222581,-0.191353,0.215282,-0.095915,-0.029505


# Call API from website

##  Using path and file name

In [80]:
import requests
path = '/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data'
file = 'Glioma-clinic-TCGA-proteins-test-with-identifier-outcome1.csv'


params = {'path': path,
          'file': file}
brainproteomics_api_url = 'http://127.0.0.1:8000/predict_one'
response = requests.get(brainproteomics_api_url, params = params)


In [81]:
response.json()

{'Outcome': 'bad cancer', 'Probability': 1.0}

## Using uploaded file

In [102]:
# load data to predict
path = "/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/raw_data"
file = "Glioma-clinic-TCGA-proteins-test-with-identifier.csv"

df = pd.read_csv(f"{path}/{file}", header=0)

X_pred = df.iloc[:,1:]

dict = {k: v.tolist() for k, v in X_pred.iterrows()}

In [103]:
X_pred.head(3)

,14-3-3_beta_p,14-3-3_epsilon_p,14-3-3_zeta_p,4E-BP1_p,4E-BP1_pS65_p,4E-BP1_pT37_T46_p,4E-BP1_pT70_p,53BP1_p,A-Raf_pS299_p,ACC1_p,...,p27_p,p27_pT157_p,p27_pT198_p,p38_pT180_Y182_p,p53_p,p62-LCK-ligand_p,p70S6K_p,p70S6K_pT389_p,p90RSK_p,p90RSK_pT359_S363_p
0,-0.046266,0.010482,0.005867,-0.128982,-0.097890,-0.286694,-0.095360,-0.315572,-0.079862,-0.190765,...,0.282454,0.215619,0.049541,0.394208,-0.173627,-0.088369,-0.160519,0.494404,0.412308,-0.081711
1,0.062830,0.341067,0.509924,-0.097006,0.050285,-0.555042,-0.108214,-0.047927,-0.029663,0.203558,...,0.233966,0.165461,0.225253,-0.065668,-0.268971,-0.059341,-0.237159,-0.004258,0.383249,-0.383407
2,0.005718,-0.049226,-0.176334,0.110042,0.084026,0.205933,-0.054664,0.562995,0.063561,0.578327,...,0.361402,-0.143755,-0.065628,0.216761,0.033749,0.105418,-0.248534,-0.251022,0.017531,-0.108484


In [89]:
pd.DataFrame(dict).transpose()

,0,1,2,3,4,5,6,7,8,9,...,164,165,166,167,168,169,170,171,172,173
0,-0.046266,0.010482,0.005867,-0.128982,-0.097890,-0.286694,-0.095360,-0.315572,-0.079862,-0.190765,...,0.282454,0.215619,0.049541,0.394208,-0.173627,-0.088369,-0.160519,0.494404,0.412308,-0.081711
1,0.062830,0.341067,0.509924,-0.097006,0.050285,-0.555042,-0.108214,-0.047927,-0.029663,0.203558,...,0.233966,0.165461,0.225253,-0.065668,-0.268971,-0.059341,-0.237159,-0.004258,0.383249,-0.383407
2,0.005718,-0.049226,-0.176334,0.110042,0.084026,0.205933,-0.054664,0.562995,0.063561,0.578327,...,0.361402,-0.143755,-0.065628,0.216761,0.033749,0.105418,-0.248534,-0.251022,0.017531,-0.108484
3,0.070416,0.068269,0.534429,-0.403894,0.106003,-0.169416,-0.234652,-0.223324,0.019363,0.397784,...,0.113838,-0.081752,-0.028470,0.059929,-0.093907,0.007829,-0.040814,0.726636,0.156300,-0.195128
4,-0.063005,0.047834,0.414731,-0.374914,0.616939,0.810131,-0.173296,-0.381965,0.101445,0.273052,...,-0.273558,-0.042558,-0.142239,-0.141052,-0.224926,-0.073704,-0.076397,1.391063,0.137433,0.104366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,0.163980,0.029452,0.649739,-0.445981,1.062340,1.745698,0.025130,-0.152001,0.239102,0.431975,...,-0.104785,0.252492,-0.155413,-0.746732,0.029633,0.219986,0.093239,2.279390,0.222581,0.341038
58,-0.029360,-0.020617,-0.058948,0.173444,0.115165,0.235074,0.065090,-0.202499,0.023789,0.080197,...,0.422843,-0.034226,0.087660,-0.013897,0.336788,0.012812,0.066353,-0.132282,-0.191353,0.017849
59,-0.082008,0.253038,0.390207,-0.166794,-0.119515,-0.566232,-0.140786,-0.212030,0.160434,0.107996,...,-0.079885,0.121889,0.062165,-0.484047,0.274336,0.177687,0.064581,1.263077,0.215282,0.108698
60,0.058965,-0.050453,0.028261,-0.050225,-0.221785,-0.788873,-0.199018,-0.101192,0.108723,-0.416462,...,-0.063589,-0.046370,0.004108,-0.000405,0.283721,0.420645,-0.148972,-0.176567,-0.095915,0.066603


In [90]:
def predict_one_upload(
            dict # "Glioma-clinic-TCGA-proteins-test-with-identifier-outcome0.csv"
            ):
    """
    Make a prediction for every row in your dataset.
    Input needs to be a csv file with rows = samples and columns = proteins.
    The first row has to contain the protein names or any other identifier that will serve as a header.
    If your file has a sample identifier column, name this column "Identifier"
    """

    X_pred = pd.DataFrame(dict).transpose()


    # Preprocess
    # Load scaler
    scaler = load_scaler(path = '/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_scalers',
                         file = 'MinMax_20240306-102844.joblib'
                        )

    X_pred_proc = scaler.transform(X_pred)

    # Predict data
    model = load_model()

    outcome_num = int(model.predict(X_pred_proc)[0])
    if outcome_num == 0:
        outcome = "good cancer"
        probability = round(float(model.predict_proba(X_pred_proc)[0][0]), 4)
    else:
        outcome = "bad cancer"
        probability = round(float(model.predict_proba(X_pred_proc)[0][1]), 4)

    return {
                "Outcome": outcome,
                "Probability": probability
    }


In [91]:
predict_one_upload(dict)

/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SGDClassifier from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
http

{'Outcome': 'bad cancer', 'Probability': 1.0}

In [ ]:
if 'Identifier' in df.columns:
    X_pred = df.drop(["Identifier"], axis = 1)
else:
    X_pred = df

dict = {k: v.tolist() for k, v in X_pred.iterrows()}

brainproteomics_api_url = 'http://127.0.0.1:8000/predict_one_upload'
response = requests.get(brainproteomics_api_url, params=dict)

prediction = response.json()

st.write(prediction)

In [92]:
dict

{0: [-0.046265886,
  0.010482387,
  0.005867445,
  -0.128982268,
  -0.097889666,
  -0.286694285,
  -0.095360044,
  -0.315571826,
  -0.079861525,
  -0.190764932,
  -0.238326955,
  -0.048540484,
  -0.002730457,
  -0.075113939,
  0.029746211,
  -0.332981871,
  -0.368910452,
  -0.0524449,
  0.287860131,
  0.488310645,
  0.206614825,
  -0.371954514,
  0.02017156,
  -0.007428026,
  -0.164302779,
  -0.189112674,
  -0.177334949,
  0.105022221,
  0.547064077,
  -0.088799185,
  0.0,
  0.02225688,
  -0.099785565,
  0.087806221,
  0.120169961,
  -0.203541575,
  0.597637885,
  -0.064870281,
  -0.006525207,
  0.493392293,
  -0.289667328,
  0.394968561,
  0.061731113,
  0.166321033,
  0.035140285,
  -0.121493066,
  -0.309331731,
  1.047156113,
  -0.384492995,
  0.068288443,
  0.15988954,
  -0.086162623,
  -0.591365353,
  -0.348480285,
  -0.242205473,
  -0.036207938,
  -0.283215889,
  -0.002190084,
  0.183297849,
  0.274069847,
  0.017332449,
  -0.171258421,
  0.03426366,
  -0.633766774,
  -0.28660364

In [104]:
def predict_one_upload(
            data # "Glioma-clinic-TCGA-proteins-test-with-identifier-outcome0.csv"
            ):
    """
    Make a prediction for every row in your dataset.
    Input needs to be a csv file with rows = samples and columns = proteins.
    The first row has to contain the protein names or any other identifier that will serve as a header.
    If your file has a sample identifier column, name this column "Identifier"
    """

    X_pred = data


    # Preprocess
    # Load scaler
    scaler = load_scaler(path = '/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_scalers',
                         file = 'MinMax_20240306-102844.joblib'
                        )

    X_pred_proc = scaler.transform(X_pred)

    # Predict data
    model = load_model()

    outcome_num = int(model.predict(X_pred_proc)[0])
    if outcome_num == 0:
        outcome = "good cancer"
        probability = round(float(model.predict_proba(X_pred_proc)[0][0]), 4)
    else:
        outcome = "bad cancer"
        probability = round(float(model.predict_proba(X_pred_proc)[0][1]), 4)

    return {
                "Outcome": outcome,
                "Probability": probability
    }


In [106]:
data = X_pred[0:1]

,14-3-3_beta_p,14-3-3_epsilon_p,14-3-3_zeta_p,4E-BP1_p,4E-BP1_pS65_p,4E-BP1_pT37_T46_p,4E-BP1_pT70_p,53BP1_p,A-Raf_pS299_p,ACC1_p,...,p27_p,p27_pT157_p,p27_pT198_p,p38_pT180_Y182_p,p53_p,p62-LCK-ligand_p,p70S6K_p,p70S6K_pT389_p,p90RSK_p,p90RSK_pT359_S363_p
0,-0.046266,0.010482,0.005867,-0.128982,-0.09789,-0.286694,-0.09536,-0.315572,-0.079862,-0.190765,...,0.282454,0.215619,0.049541,0.394208,-0.173627,-0.088369,-0.160519,0.494404,0.412308,-0.081711


In [107]:
predict_one_upload(X_pred[0:1])

/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SGDClassifier from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'Outcome': 'bad cancer', 'Probability': 1.0}

In [108]:
model = load_model()
model.predict(X_pred)

/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SGDClassifier from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jana/.pyenv/versions/3.10.6/envs/brain_proteomics/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


array([0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1])

In [116]:
def load_scaler(path = '/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_scalers/',
               file = 'scaler.joblib'
              ):
    """
    Loads a locally saved and fitted scaler from the given path and file name.
    """
    path_file = f"{path}/{file}"

    scaler = load(path_file)
    return scaler

In [121]:
os.getcwd()

'/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/jfschulz_notebooks'

In [119]:
load_scaler(path = '/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_scalers',
            file = 'scaler.joblib')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jana/code/Klara-haas/brain_proteomics_project/brain_proteomics/api/saved_scalers/scaler.joblib'